# GameDay2020 Notebook

The standard dilemma of fantasy baseball is who to choose next: the best available player but in a position you've already filled?; the best at a rare position?; the next best closer because they're going fast?  Whatever choice you make will have consequences, as the next player will no longer have your pick as a choice, and so on.  Ideally you would forecast how each of those would play out and take the best option --- that's exactly what this code does: it identifies the best player left in each of the positions you still need to fill, simulates the rest of draft in each of those scenarios, and returns the best choice.

And it's easy to use!  After [cloning](https://github.com/wrapgenius/GameDay2020) GameDay2020 (and provided you have Jupyter Notebooks with Python 3 installed, and a spreadsheet program) you should have everything you need to do a live roto draft.

The code is based on three ingredients: *projections*, *rankings*, and *injuries*:  
- Projections are estimates of player performances for 2020, and are based on data scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips) (model = ZiPS, Steamer, or TheBat.) 
- Rankings are draft-order recommendations.  The default (i.e.; ranking_method = 'Yahoo') is to use Yahoo Rankings (from Feb 22, 2020).  Others include [ESPN](https://www.espn.com/fantasy/baseball/story/_/id/28285423/fantasy-baseball-2020-rankings-roto-rotisserie), [FantasyPros](https://www.fantasypros.com/mlb/rankings/overall.php), and [Rotographs](https://fantasy.fangraphs.com), or you can put in your own.  Yahoo for yahoo leagues makes most sense since it most closely imitates autodraft.    
- Injured players to be removed from draft consideration are located in the Injuries2020.xlsx file found in the Injured_List_Spreadsheets directory. 

_Your_ roto league may have more or less than 12 teams, or use different stats; that ok!  Declare them when defining the Draft object: most of the obvious stats are included, more obscure ones may require hacking into the [fangraphs_projection_2020](https://github.com/wrapgenius/GameDay2020/blob/master/GameDayFunctions/fangraphs_projection_2020.py).  

Note, if you choose to shuffle_picks = True, it will not default to the next best pick, but draw a random number between 0 and 1 and compare it to the sigmoid function 1/1+exp(-(pick_number - avg_pick_number)).  So say it's the 15th pick and Trevor Story, who is typically drafted 10th is available, there is an 85% chance you'll select him.  Also note, it can't _yet_ do AL or NL only. 

Below are examples of the different ways the code can be run.  

In [1]:
# Load packages.  Must have pandas and numpy.  

import pdb
import os
import sys
import pandas as pd
pd.options.mode.chained_assignment = None
import numpy as np
import copy
from GameDayFunctions.fangraphs_projection_2020 import Projection
from GameDayFunctions.draft_2020 import Draft

In [2]:
year = 2020
path_data = "projections/"
path_drafts = "Draft_Pick_Spreadsheets/"
shuffle_picks = False

First need a **Projection** object.  Projections are scraped from [Fangraphs](https://www.fangraphs.com/projections.aspx?pos=all&stats=bat&type=zips), and include ZiPS, Steamer, and TheBat.  
In this example I create a dictionary of the three projections.  

In [3]:
player_ranking_zips = Projection(path_data=path_data,year=year,model='ZiPS',ranking_method = 'Yahoo')
player_ranking_steam = Projection(path_data=path_data,year=year,model='Steamer',ranking_method = 'FantasyPros')
player_ranking_thebat = Projection(path_data=path_data,year=year,model='TheBat',ranking_method = 'ESPN')
player_rankings = {"ZiPS":player_ranking_zips, "Steamer":player_ranking_steam, "TheBat":player_ranking_thebat}

Next need a **Draft** object.  The default, e.g.; Draft(player_rankings['ZiPS']), uses 12 teams and the roster spots/roto stats shown in this example. 

In [4]:
# Enter your roto-league-specific stats.  The default settings are shown here.  
draft_position = 2
number_teams = 12
roster_spots = {'C':1,'1B':1,'2B':1, '3B':1,'SS':1,'OF':3,'UTIL':1,'SP':2,'RP':2,'P':3,'BN':5}
batter_stats  = ['AB','R','1B','2B', '3B','HR','RBI','SB','BB','AVG','OPS']
pitcher_stats = ['IP','W', 'L','CG','SHO','SV','BB','SO','ERA','WHIP','BSV'] 

# Choose the projection system you prefer ()
player_projections = player_rankings['ZiPS']

# Get an instance of the Draft Class with your league-specific details and projection preference.
zips_example = Draft(player_projections, 
                     draft_position = draft_position, 
                     number_teams = number_teams,
                     roster_spots = roster_spots,
                     batter_stats = batter_stats,
                     pitcher_stats = pitcher_stats)

To use while drafting in real-time, simply record the draft results in a spreadsheet (.xlsx, so if you're using Numbers, need to export to Excel) and run the cell. 

Examples below show results for after 2, 3, and 4 rounds.  Notice that at each stage the projected placement in the standings improves.  

In [6]:
# Example 5th pick in the 2nd round

zipsT = Draft(player_rankings['ZiPS'], draft_position = 5)
player_to_pick, projected_roto_stats = zipsT.draft_from_list_and_find_best_pick(search_depth = 2, draft_pick_file = 'TestPicksround2.xlsx', path_list = path_drafts)

Finding Best Pick For Team 5
Stored Result for Pick 0 [20/12] J.D. Martinez OF whose placement/score is 11/83.5
Stored Result for Pick 1 [20/21] Xander Bogaerts SS whose placement/score is 11/82.5
Stored Result for Pick 2 [20/22] Jack Flaherty SP whose placement/score is 10/86.5
Stored Result for Pick 3 [20/23] Walker Buehler SP whose placement/score is 10/87.5
Stored Result for Pick 4 [20/24] Ketel Marte 2B whose placement/score is 10/101.0
Stored Result for Pick 5 [20/25] Rafael Devers 3B whose placement/score is 9/97.0
Stored Result for Pick 6 [20/26] Trea Turner SS whose placement/score is 10/88.5
Stored Result for Pick 7 [20/27] Roberto Osuna RP whose placement/score is 10/89.5
Stored Result for Pick 8 [20/28] Bryan Reynolds OF whose placement/score is 11/80.5
Stored Result for Pick 9 [20/30] Jose Ramirez 3B whose placement/score is 9/121.0
Stored Result for Pick 10 [20/31] Ozzie Albies 2B whose placement/score is 10/109.0
Stored Result for Pick 11 [20/33] Josh Hader RP whose plac

In [7]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = projected_roto_stats[0]
projected_standings = projected_roto_stats[3]
projected_place = projected_roto_stats[4]
projected_scores = projected_roto_stats[5]

In [8]:
# The projected_stats give the raw numbers

projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,758.0,692.0,261.0,28.0,258.0,804.0,72.0,546.0,0.267912,0.853736,48.0,34.0,3.0,3.0,37.0,194.0,861.0,3.593102,1.136447,3.0
1,755.0,791.0,257.0,23.0,206.0,703.0,141.0,515.0,0.271101,0.813117,57.0,37.0,4.0,4.0,25.0,296.0,1084.0,3.403657,1.142218,2.0
2,730.0,846.0,269.0,30.0,195.0,706.0,91.0,476.0,0.276714,0.813460,58.0,36.0,3.0,3.0,76.0,199.0,851.0,3.621034,1.135528,6.0
3,649.0,727.0,234.0,27.0,181.0,620.0,100.0,425.0,0.274143,0.811843,49.0,32.0,2.0,2.0,54.0,242.0,786.0,3.689254,1.199941,4.0
4,703.0,811.0,242.0,25.0,189.0,677.0,51.0,460.0,0.281927,0.824897,49.0,30.0,2.0,2.0,52.0,224.0,813.0,3.443559,1.171118,5.0
5,613.0,631.0,221.0,20.0,205.0,681.0,38.0,453.0,0.271673,0.839852,53.0,35.0,3.0,3.0,23.0,247.0,907.0,3.620877,1.190937,2.0
6,683.0,801.0,264.0,24.0,198.0,699.0,124.0,479.0,0.276326,0.817695,50.0,30.0,3.0,3.0,40.0,249.0,951.0,3.355441,1.133794,3.0
7,809.0,809.0,267.0,41.0,233.0,784.0,75.0,541.0,0.277245,0.844384,36.0,32.0,0.0,0.0,29.0,236.0,575.0,4.077550,1.336144,2.0
8,619.0,585.0,219.0,25.0,202.0,638.0,105.0,413.0,0.271590,0.850285,50.0,35.0,1.0,1.0,24.0,225.0,817.0,3.685763,1.197386,2.0
9,536.0,644.0,185.0,22.0,153.0,552.0,65.0,355.0,0.273769,0.803421,59.0,33.0,2.0,2.0,115.0,239.0,974.0,3.271082,1.119879,9.0


In [9]:
# And projected_scores show how they translate to roto scores

projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,11.0,5.0,8.0,9.5,12.0,12.0,4.0,12.0,2.0,12.0,3.0,5.0,9.0,9.0,6.0,12.0,7.0,7.0,8.0,6.5
1,10.0,8.0,7.0,3.0,9.0,8.0,12.0,10.0,4.0,3.0,9.5,1.0,12.0,12.0,4.0,1.0,12.0,10.0,7.0,10.0
2,8.0,12.0,11.0,11.0,5.0,9.0,8.0,7.0,10.0,4.0,11.0,2.0,9.0,9.0,11.0,11.0,6.0,5.0,9.0,2.0
3,5.0,6.0,5.0,8.0,3.0,3.0,9.0,4.0,8.0,2.0,4.5,8.5,5.0,5.0,10.0,4.0,3.0,3.0,3.0,4.5
4,7.0,11.0,6.0,6.5,4.0,5.0,2.0,6.0,12.0,8.0,4.5,11.5,5.0,5.0,9.0,10.0,4.0,8.0,6.0,3.0
5,3.0,3.0,4.0,1.0,8.0,6.0,1.0,5.0,6.0,9.0,8.0,3.5,9.0,9.0,1.5,3.0,8.0,6.0,5.0,10.0
6,6.0,9.0,9.0,4.5,6.0,7.0,11.0,8.0,9.0,5.0,6.5,11.5,9.0,9.0,7.0,2.0,9.0,11.0,10.0,6.5
7,12.0,10.0,10.0,12.0,10.0,11.0,5.5,11.0,11.0,10.0,1.0,8.5,1.0,1.0,5.0,7.0,1.0,1.0,1.0,10.0
8,4.0,1.0,3.0,6.5,7.0,4.0,10.0,3.0,5.0,11.0,6.5,3.5,2.5,2.5,3.0,9.0,5.0,4.0,4.0,10.0
9,1.0,4.0,1.0,2.0,1.0,2.0,3.0,1.0,7.0,1.0,12.0,6.5,5.0,5.0,12.0,5.5,10.0,12.0,11.0,1.0


In [10]:
# Finally, the projected_standings shows the resulting standings

projected_standings

10    164.0
2     160.0
0     160.0
6     156.0
1     152.5
7     139.0
4     133.5
5     109.0
8     104.5
3     103.5
9     103.0
11     75.0
dtype: float64

In [11]:
# But if you just want to know how your team finishes, that's projected_place

projected_place

7

### You can simulate drafts as well. Naive drafts simply picks the next eligible (i.e., open roster spot) player on the ranked list. 

In [12]:
# Try a quick draft: 4 teams, limited roster spots, naive (i.e., next available pick on ranked list) picks.

naive_draft = True
zipsS= Draft(player_rankings['ZiPS'],draft_position = 2,number_teams = 4, roster_spots = {'C':0,'1B':0,'2B':1, '3B':1,'SS':1,'OF':2,'UTIL':1,'SP':1,'RP':1,'P':1,'BN':1})
zipsS.draft_all(naive_draft = naive_draft, shuffle_picks = shuffle_picks)
print('Placement = '+str(zipsS.roto_placement))

Placement = 3


In [13]:
# Now try a naive draft again, but for full 12 team and all 24 roster spots.  Draft Position is 2 

naive_draft = True
zipsNaive_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zipsNaive_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_2.roto_placement))

Placement  = 6


In [14]:
zipsNaive_2.teams[1]['roster']

{'OF': [[array(['Mookie Betts'], dtype=object),
   array(['Aaron Judge'], dtype=object)],
  array(['Lorenzo Cain'], dtype=object)],
 'SP': [array(['Walker Buehler'], dtype=object),
  array(['Tyler Glasnow'], dtype=object)],
 'SS': array(['Trea Turner'], dtype=object),
 '1B': array(['Anthony Rizzo'], dtype=object),
 'RP': [array(['Will Smith'], dtype=object),
  array(['Ryan Pressly'], dtype=object)],
 'P': [[array(['Luis Castillo'], dtype=object),
   array(['Tommy Kahnle'], dtype=object)],
  array(['Chris Martin'], dtype=object)],
 'UTIL': array(['Matt Olson'], dtype=object),
 '2B': array(['Kevin Newman'], dtype=object),
 '3B': array(['David Fletcher'], dtype=object),
 'C': array(['Yadier Molina'], dtype=object),
 'BN': array(['Tony Gonsolin'], dtype=object)}

In [ ]:
# This time do the full-simulation, i.e., with naive_draft = False
# Remember, it will try every remaining open position and pick the one whose resulting placement is highest.  
# Setting silent = False shows the decision logic for each pick (highest placement followed by highest score [placement, score])

naive_draft = False
zips_2 = Draft(player_rankings['ZiPS'], draft_position = 2)
zips_2.draft_all(naive_draft = naive_draft, search_depth = 2, shuffle_picks = shuffle_picks,silent=False)
print('Placement  = '+str(zips_2.roto_placement))

Team 1 Drafting Christian Yelich
Stored Result for Pick 0 [2/2] Mookie Betts OF whose placement/score is 6/138.5
Stored Result for Pick 1 [2/3] Mike Trout OF whose placement/score is 6/137.0
Stored Result for Pick 2 [2/6] Anthony Rendon 3B whose placement/score is 4/132.0
Stored Result for Pick 3 [2/7] Cody Bellinger 1B whose placement/score is 4/142.0
Stored Result for Pick 4 [2/8] Freddie Freeman 1B whose placement/score is 5/137.0
Stored Result for Pick 5 [2/9] Alex Bregman 3B whose placement/score is 4/149.0
Stored Result for Pick 6 [2/11] Justin Verlander SP whose placement/score is 4/137.0
Stored Result for Pick 7 [2/13] Jacob deGrom SP whose placement/score is 4/137.0
Stored Result for Pick 8 [2/14] Francisco Lindor SS whose placement/score is 5/138.0
Stored Result for Pick 9 [2/15] Whit Merrifield 2B whose placement/score is 5/134.0
Stored Result for Pick 10 [2/17] Trevor Story SS whose placement/score is 3/142.0
Stored Result for Pick 11 [2/19] Jose Altuve 2B whose placement/s

In [ ]:
zips_2.teams[1]

In [14]:
# Projected_roto_stats contains the estimated results from all teams given the recommented pick

projected_stats = zips_2.roto_team_stats
projected_standings = zips_2.roto_standings
projected_place = zips_2.roto_placement
projected_scores = zips_2.roto_team_stats_rank

In [15]:
projected_place

1

In [16]:
projected_stats

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BB,SO,ERA,WHIP,BSV
0,720.0,790.0,262.0,28.0,219.0,745.0,98.0,492.0,0.272076,0.821402,55.0,37.0,4.0,4.0,27.0,241.0,1008.0,3.588769,1.146309,2.0
1,929.0,995.0,320.0,42.0,265.0,913.0,116.0,546.0,0.276151,0.822852,46.0,29.0,3.0,3.0,50.0,188.0,780.0,3.498711,1.124196,5.0
2,769.0,848.0,265.0,33.0,193.0,669.0,149.0,490.0,0.275204,0.807540,47.0,35.0,2.0,2.0,17.0,241.0,794.0,3.772834,1.205395,1.0
3,627.0,620.0,224.0,23.0,196.0,671.0,50.0,460.0,0.265500,0.825660,67.0,41.0,4.0,4.0,31.0,312.0,1143.0,3.495938,1.154816,3.0
4,506.0,573.0,178.0,13.0,140.0,501.0,47.0,379.0,0.279400,0.828933,41.0,29.0,2.0,2.0,67.0,194.0,670.0,3.833686,1.209472,5.0
5,628.0,675.0,222.0,22.0,198.0,667.0,94.0,464.0,0.271670,0.829042,36.0,22.0,1.0,1.0,42.0,216.0,707.0,3.509932,1.187927,3.0
6,721.0,797.0,262.0,24.0,227.0,749.0,74.0,526.0,0.274110,0.830180,35.0,24.0,1.0,1.0,50.0,186.0,604.0,3.501016,1.206735,4.0
7,710.0,752.0,252.0,31.0,214.0,734.0,87.0,451.0,0.274532,0.828095,56.0,33.0,3.0,3.0,23.0,243.0,968.0,3.434494,1.150922,2.0
8,740.0,764.0,261.0,28.0,215.0,740.0,88.0,514.0,0.273117,0.829282,60.0,39.0,2.0,2.0,67.0,277.0,1003.0,3.571324,1.180071,5.0
9,782.0,744.0,268.0,23.0,260.0,827.0,94.0,611.0,0.271100,0.856808,45.0,32.0,1.0,1.0,53.0,218.0,723.0,3.615973,1.220705,4.0


In [17]:
projected_scores

,R,1B,2B,3B,HR,RBI,SB,BB,AVG,OPS,W,L,CG,SHO,SV,BBP,SO,ERA,WHIP,BSV
0,7.0,9.0,8.5,8.0,9.0,9.0,10.0,8.0,5.0,3.0,8.0,4.0,11.5,11.5,4.0,4.5,11.0,5.0,10.0,10.0
1,12.0,12.0,12.0,12.0,12.0,12.0,11.0,11.0,11.0,4.0,5.0,9.5,9.0,9.0,7.5,11.0,5.0,9.0,12.0,3.0
2,10.0,11.0,10.0,11.0,3.0,5.0,12.0,7.0,9.0,2.0,6.0,5.0,6.0,6.0,1.0,4.5,7.0,2.0,4.0,12.0
3,3.0,3.0,4.0,3.5,4.0,6.0,2.5,5.0,1.0,5.0,12.0,1.0,11.5,11.5,5.0,1.0,12.0,10.0,8.0,7.5
4,1.0,1.0,1.0,1.0,1.0,1.0,1.0,2.0,12.0,8.0,3.0,9.5,6.0,6.0,10.5,10.0,2.0,1.0,2.0,3.0
5,4.0,4.0,3.0,2.0,5.0,4.0,8.5,6.0,4.0,9.0,2.0,12.0,2.5,2.5,6.0,9.0,3.0,7.0,5.0,7.5
6,8.0,10.0,8.5,5.0,10.0,10.0,4.0,10.0,7.0,11.0,1.0,11.0,2.5,2.5,7.5,12.0,1.0,8.0,3.0,5.5
7,6.0,7.0,6.0,10.0,7.0,7.0,6.0,4.0,8.0,7.0,9.0,7.0,9.0,9.0,2.0,3.0,9.0,11.0,9.0,10.0
8,9.0,8.0,7.0,8.0,8.0,8.0,7.0,9.0,6.0,10.0,10.0,2.5,6.0,6.0,10.5,2.0,10.0,6.0,6.0,3.0
9,11.0,6.0,11.0,3.5,11.0,11.0,8.5,12.0,3.0,12.0,4.0,8.0,2.5,2.5,9.0,8.0,4.0,4.0,1.0,5.5


In [18]:
projected_standings

1     189.0
0     156.0
7     146.0
8     142.0
9     137.5
6     137.5
2     133.5
3     116.5
10    108.5
5     106.0
11    105.5
4      82.0
dtype: float64

In [19]:
# Lets compare Naive True/False for the 7th pick

naive_draft = True
zipsNaive_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zipsNaive_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_7.roto_placement))

Placement  = 8


In [20]:
naive_draft = False
zips_7 = Draft(player_rankings['ZiPS'], draft_position = 7)
zips_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_7.roto_placement))

Placement  = 1


In [21]:
naive_draft = False
steamer_7 = Draft(player_rankings['Steamer'], draft_position = 7)
steamer_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steamer_7.roto_placement))

Placement  = 1


In [22]:
# And for the 11th pick

naive_draft = True
zipsNaive_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zipsNaive_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zipsNaive_11.roto_placement))

Placement  = 7


In [23]:
naive_draft = False
zips_11 = Draft(player_rankings['ZiPS'], draft_position = 11)
zips_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(zips_11.roto_placement))

Placement  = 1


In [29]:
# We can do simlated drafts for different projection systems.  Above was all with ZiPs.  Lets try a Steamer, with custom league settings.

number_teams = 6
naive_draft  = True
steam_custom = Draft(player_rankings['ZiPS'])
steam_custom.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Placement  = 6


In [30]:
# And compare Naive to the recommendations from the simulation

naive_draft = False
steam_custom = Draft(player_rankings['ZiPS'], number_teams=number_teams)
steam_custom.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_custom.roto_placement))

Placement  = 1


In [31]:
# And lets compare Naive True/False for different draft positions

naive_draft = True
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Placement  = 10


In [32]:
naive_draft = False
steam_2= Draft(player_rankings['Steamer'], draft_position = 2)
steam_2.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_2.roto_placement))

Placement  = 1


In [33]:
naive_draft = True
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Placement  = 5


In [34]:
naive_draft = False
steam_7= Draft(player_rankings['Steamer'], draft_position = 7)
steam_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_7.roto_placement))

Placement  = 1


In [42]:
steam_7.teams[7 - 1]

{'batting_stats':         AB    R   1B  2B 3B  HR  RBI  SB  BB    AVG    OPS
 6184   570  101   97  34  2  39  119   3  69  0.301  0.954
 11739  504   71   81  29  2  24   77   6  39  0.270  0.806
 11493  591   93   92  29  3  37  102   7  63  0.272  0.862
 19612  603   92   99  42  3  22   74  24  45  0.275  0.792
 5038   538  100   77  30  1  36  103   4  93  0.267  0.906
 11368  452   74   61  20  2  25   73   3  82  0.239  0.817
 6153   590   80   88  32  7  25   87   4  49  0.257  0.779
 15149  588   86   97  31  2  30   90   1  54  0.272  0.824
 14225  418   58   68  21  2  21   68  12  41  0.268  0.814
 14161  481   71   72  31  4  19   71  23  36  0.262  0.779
 19955  477   62   83  23  4  17   65   9  44  0.266  0.766
 19326  585   81  113  34  4  18   73   7  53  0.288  0.804,
 'pitching_stats':         IP   W   L CG SHO  SV  BB   SO   ERA  WHIP BSV
 0    205.0  14   9  2   2   0  41  258  3.04  1.02   0
 6    201.0  14   9  1   1   0  31  218  3.57  1.08   0
 203   65.0   3 

In [35]:
naive_draft = True
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Placement  = 1


In [36]:
naive_draft = False
steam_11= Draft(player_rankings['Steamer'], draft_position = 11)
steam_11.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(steam_11.roto_placement))

Placement  = 1


In [40]:
steam_11.teams[11 - 1]['roster']

{'OF': [[[[array(['Juan Soto'], dtype=object),
     array(['Charlie Blackmon'], dtype=object)],
    array(['Eloy Jimenez'], dtype=object)],
   array(['Bryan Reynolds'], dtype=object)],
  array(['Andrew McCutchen'], dtype=object)],
 'SP': [array(['Justin Verlander'], dtype=object),
  array(['Brandon Woodruff'], dtype=object)],
 '3B': [array(['Kris Bryant'], dtype=object),
  array(['Yoan Moncada'], dtype=object)],
 'RP': [array(['Roberto Osuna'], dtype=object),
  array(['Taylor Rogers'], dtype=object)],
 '2B': [[array(['Jeff McNeil'], dtype=object),
   array(['Tommy Edman'], dtype=object)],
  array(['Garrett Hampson'], dtype=object)],
 'SS': [array(['Jorge Polanco'], dtype=object),
  array(['Amed Rosario'], dtype=object)],
 '1B': array(['Miguel Sano'], dtype=object)}

In [37]:
# And finally, 7th pick using TheBat projection system. 

naive_draft = True
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Placement  = 11


In [38]:
naive_draft = False
thebat_7= Draft(player_rankings['TheBat'], draft_position = 7)
thebat_7.draft_all(naive_draft = naive_draft,shuffle_picks = shuffle_picks)
print('Placement  = '+str(thebat_7.roto_placement))

Placement  = 1
